In [ ]:
# FEATURE ENGINEERING FOR TRANSFORMER TIMESERIES

In [3]:
from workalendar.europe import Germany
from datetime import date, timedelta
import pandas as pd



# Create a Germany calendar instance
germany_calendar = Germany()

# Define the range of years for which you want to get the holidays
start_year = 2019
end_year = 2023

# Initialize an empty DataFrame to store holidays
holidays_df = pd.DataFrame(columns=['date', 'holiday_name'])

# Get holidays for each year and append to the DataFrame
for year in range(start_year, end_year + 1):
    holidays = germany_calendar.holidays(year)
    year_holidays_df = pd.DataFrame(holidays, columns=['date', 'holiday_name'])
    holidays_df = pd.concat([holidays_df, year_holidays_df], ignore_index=True)

# Add weekends (Saturdays and Sundays) to the DataFrame
current_date = date(start_year, 1, 1)
end_date = date(end_year, 12, 31)

weekend_data = []

while current_date <= end_date:
    if current_date.weekday() in [5, 6]:  # 5 is Saturday, 6 is Sunday
        weekend_data.append({'date': current_date, 'holiday_name': 'Weekend'})
    current_date += timedelta(days=1)

# Convert the list to a DataFrame
weekend_df = pd.DataFrame(weekend_data)

# Append the weekend DataFrame to the holidays DataFrame
holidays_df = pd.concat([holidays_df, weekend_df], ignore_index=True)

holidays_df = holidays_df.sort_values(by='date')

# Print the sorted DataFrame
print(holidays_df)

           date          holiday_name
0    2019-01-01              New year
45   2019-01-05               Weekend
46   2019-01-06               Weekend
47   2019-01-12               Weekend
48   2019-01-13               Weekend
..          ...                   ...
564  2023-12-24               Weekend
43   2023-12-25         Christmas Day
44   2023-12-26  Second Christmas Day
565  2023-12-30               Weekend
566  2023-12-31               Weekend

[567 rows x 2 columns]


In [ ]:
data = pd.read_csv("./timeseries_data/trafo_67.csv")
data["timestamp"] = pd.to_datetime(data["timestampUtc"])
data_modified = data.drop(columns=["timestampUtc"]).set_index("timestamp")




data_modified_fe = data_modified.copy()

# Encoding local timestamp
data_modified_fe['Day_value_Stamp'] = data_modified.index.hour * 4 + data_modified.index.minute / 15

# Indicator variable for bridgedays (Mondays in case of a holiday on Tuesday)
#df['is_bridgeday'] = ((df['timestamp'].dt.dayofweek == 0) & ((df['timestamp'] + timedelta(days=1)).dt.isin(holiday_dates)))

# Day of the week
data_modified_fe['day_of_week'] = data_modified.index.dayofweek


# Convert index to pandas Series of dates
data_dates = pd.to_datetime(data_modified_fe.index.date)

# Check if each date is in the holidays DataFrame
data_modified_fe['Holidays'] = data_dates.isin(holidays_df['date']).astype(np.int8)



#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# Print the resulting DataFrame
print(data_modified_fe)

In [ ]:
from datetime import date
import holidays

germany_holidays = holidays.DE(subdiv='BW')  



# Assuming data_modified_fe is your DataFrame with a DatetimeIndex
data_dates = pd.to_datetime(data_modified_fe.index.date)

data_modified_fe_1 = data_modified_fe.copy()

# Check if each date is a holiday in the 'BW' region
data_modified_fe_1['Holidays'] = data_dates.isin(germany_holidays)

# Convert boolean to integers (0 for False, 1 for True)
data_modified_fe_1['Holidays'] = data_modified_fe_1['Holidays'].astype(np.int8)

# Print the resulting DataFrame
print(data_modified_fe_1)